In [ ]:
%pip install tqdm

In [ ]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
from os import path
from sys import path as sys_path
parent_dir = path.abspath(path.join('..'))
if parent_dir not in sys_path:
    sys_path.append(parent_dir)
    print(f"Added {parent_dir.split("\\")[-1]} to sys.path")
from models.commit import Commit
from models.file import File
from utils.worker import get_optimal_max_workers

In [ ]:
commits = Commit.fetch_all_commits()
parent_folder = path.join('..', 'download', 'orgs')
max_workers = get_optimal_max_workers()

In [ ]:
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = {executor.submit(Commit.get_file_names_from_commit, path.join(parent_folder, com.org_name, com.repo_name), com.sha): com for com in commits}
    new_files_candidates = []
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing files"):
        com = futures[future]
        try:
            file_names = future.result()
            if file_names:
                for name in file_names:
                    file_data = File(name, com.repo_name, com.org_name, name.split('.')[-1].lower())
                    new_files_candidates.append(file_data)
        except Exception as e:
            print(f"Error processing file {com.sha}: {e}")
    if new_files_candidates:
        exist_file_flags = File.exists_in_batches(new_files_candidates)
        new_files = [f for f, exists in zip(new_files_candidates, exist_file_flags) if not exists]

        if new_files:
            File.add_files_in_batches(new_files)